In [1]:
from pyspark import SparkContext
import csv
from scipy import linalg, sparse
import numpy as np
from itertools import groupby
from functools import reduce
from operator import itemgetter
from collections import defaultdict
from itertools import combinations, permutations
import random
import pdb


sc = SparkContext.getOrCreate()

In [2]:

train_rdd = sc.textFile("data/train.csv")
icm_rdd = sc.textFile("data/icm_fede.csv")
test_rdd= sc.textFile("data/target_users.csv")

train_header = train_rdd.first()
icm_header = icm_rdd.first()
test_header= test_rdd.first()

train_clean_data = train_rdd.filter(lambda x: x != train_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1]), float(x[2])))
icm_clean_data = icm_rdd.filter(lambda x: x != icm_header).map(lambda line: line.split(',')).map(lambda x: (int(x[0]), int(x[1])))
test_clean_data= test_rdd.filter(lambda x: x != test_header).map(lambda line: line.split(','))



In [3]:

test_users=test_clean_data.map( lambda x: int(x[0])).collect()
#test_users=[1,2,3,4]
#test_users.take(10)

#for every item all its features
#rouped_features = sc.parallelize([(1,[1,2]),(2,[2,3,4]),(3,[3,4]),(4,[1,2,4])])
grouped_features = icm_clean_data.map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1])))
#grouped_features.take(10)
grouped_features.cache()
total_items = grouped_features.count()
grouped_features_arr = grouped_features.collect()
grouped_features_dic = sc.broadcast(dict(grouped_features.collect()))

grouped_features.map(lambda x: (x[0], x[1], 1/ np.sqrt(len(x[1])))).map(lambda x: (x[0], [(item, x[2]) for item in x[1]]))

tf_grouped_features = grouped_features.map(lambda x: (x[0], x[1], 1/ np.sqrt(len(x[1])))).map(lambda x: (x[0], [(item, x[2]) for item in x[1]]))
tf_grouped_features_dic = sc.broadcast(dict(tf_grouped_features.collect()))
tf_grouped_features_dic.value.get(11812)

tf_item = tf_grouped_features.map(lambda x: (x[0], x[1][0][1])).collect()
tf_item_dic = dict(tf_item)

#for every features all its items
#grouped_items = sc.parallelize([(1,[1,4]),(2,[1,2,4]),(3,[2,3]),(4,[2,3,4])])
grouped_items = icm_clean_data.map(lambda x: (x[1], x[0])).groupByKey().map(lambda x: (x[0], list(x[1])))
grouped_items.take(10)
grouped_items.cache()
grouped_items_dic = dict(grouped_items.collect())

idf_features = sc.broadcast(dict(grouped_items.map(lambda x: (x[0], np.log10(total_items / len(x[1])))).collect()))

def group_items_tf(f_items):
    feature = f_items[0]
    items = f_items[1]
    return (feature, [(i, tf_item_dic.get(i, 0)) for i in items])
tf_grouped_items = dict(grouped_items.map(group_items_tf).collect())

#for every user all its ratings (item, rate)
#grouped_rates = sc.parallelize([(1,[(1,8),(3,2)]),(2,[(1,2),(2,9),(3,7)]),(3,[(3,1),(4,10)])])
grouped_rates = train_clean_data.map(lambda x: (x[0],(x[1], x[2]))).groupByKey().map(lambda x: (x[0], list(x[1])))
grouped_rates.cache()
grouped_rates_dic = dict(train_clean_data.map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect())
#for every item all its ratings
item_ratings = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
#item_ratings.take(10)
shrinkage_factor = 5
item_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()
#.map(lambda x: x[0])
#return only test users
def is_in_test(user):
    return user[0] in test_users

test_user_ratings = grouped_rates.filter(is_in_test).sortByKey()
test_user_ratings.cache()

test_voted_items = test_user_ratings.map(lambda x: (x[0], [item for item, rate in x[1]])).collect()
test_voted_items_dic = dict(test_voted_items)

test_user_features = grouped_rates.map(lambda x: (x[0], [grouped_features_dic.value.get(item, []) for item, rating in x[1]])).map(lambda x: (x[0], set(reduce(lambda x,y: x+y, x[1]))))
test_user_features_dic = sc.broadcast(dict(test_user_features.collect()))


In [4]:
print(total_items)
print(idf_features.value.get(1))
print(grouped_rates_dic[2])

36797
3.61156990335
[7758]


In [5]:

#test_user_features_dic.value
def calculate_ratings(user_rates):
    user_id = user_rates[0]
    i_rates = user_rates[1]
    result = list()
    for item, rating in i_rates:
        result += list(map(lambda x: (x[0], rating * x[1]), tf_grouped_features_dic.value.get(item, [])))
    result.sort(key = lambda x: x[0])
    result = [(x,sum([z[1] for z in y])) for x,y in groupby(result, itemgetter(0))]
    return (user_id, result)

def calculate_user_idf(user_tf):
    user_id = user_tf[0]
    ratings = user_tf[1]
    result = list()
    for feature, rating in ratings:
        result += [(feature, rating * idf_features.value.get(feature, 0))]
    return(user_id, result)

def calculate_final_percentages(user_tf, n):
    user_id = user_tf[0]
    #print(user_id)
    ratings = user_tf[1]
    items_dict = defaultdict(int)
    already_voted = grouped_rates_dic[user_id]
    for feature, rating in ratings:
        items_with_f = tf_grouped_items.get(feature, [])
        for item, tf in items_with_f:
            items_dict[item] += tf * rating

    scored_items = [(total,item) for item,total in items_dict.items() if total != 0 and not item in already_voted]

    # sort the scored items in ascending order
    scored_items.sort(reverse=True)

    # take out the item score
    #ranked_items = [x[1] for x in scored_items]
    ranked_items = scored_items
    if n == -1:
        return user_id,ranked_items
    return user_id,ranked_items[:n]


In [6]:
users_tf = test_user_ratings.map(calculate_ratings)

In [7]:


users_final_ratings = users_tf.map(calculate_user_idf).map(lambda x: calculate_final_percentages(x, -1))
users_final_ratings.take(1)



[(4,
  [(10.645831795733729, 32578),
   (10.146160793559531, 35061),
   (8.3772621380919716, 98),
   (8.0441798186082032, 30408),
   (7.8069883454017415, 10129),
   (7.7066655528832948, 20424),
   (7.6960727495559551, 26942),
   (7.6650547211980777, 4709),
   (7.6538866566445796, 9793),
   (7.6057482402596683, 37030),
   (7.6057482402596683, 8482),
   (7.5122174067043979, 14572),
   (7.2852281083648496, 29330),
   (7.1757742394970894, 36637),
   (7.1757742394970894, 5547),
   (7.1251788408507082, 30892),
   (7.1251788408507082, 5176),
   (7.070271708751152, 10577),
   (6.9848502373041912, 692),
   (6.9397501354380235, 2148),
   (6.8272779014194729, 29307),
   (6.7610502341669489, 23332),
   (6.7610502341669489, 17913),
   (6.7100875814661398, 27004),
   (6.6943740024694867, 8261),
   (6.6943740024694867, 8077),
   (6.6943740024694867, 652),
   (6.683781199142147, 16141),
   (6.683781199142147, 7961),
   (6.683781199142147, 3505),
   (6.6612388585330429, 34168),
   (6.6350675215310773, 

In [8]:
def parseVector(line):
    '''
    Parse each line of the specified data file, assuming a "|" delimiter.
    Converts each rating to a float
    '''
    line = line.split(",")
    return int(line[0]),int(line[1]),float(line[2])

def parseVectorOnUser(line):
    '''
    Parse each line of the specified data file, assuming a "|" delimiter.
    Key is user_id, converts each rating to a float.
    '''
    line = line.split(",")
    return line[0],(line[1],float(line[2]))

def parseVectorOnItem(line):
    '''
    Parse each line of the specified data file, assuming a "|" delimiter.
    Key is item_id, converts each rating to a float.
    '''
    line = line.split(",")
    return line[1],(line[0],float(line[2]))

In [9]:


def sampleInteractions(item_id,users_with_rating,n):
    '''
    For items with # interactions > n, replace their interaction history
    with a sample of n users_with_rating
    '''
    if len(users_with_rating) > n:
        return item_id, random.sample(users_with_rating,n)
    else:
        return item_id, users_with_rating

def findUserPairs(item_id,users_with_rating):
    '''
    For each item, find all user-user pairs combos. (i.e. users with the same item)
    '''
    result = list()
    for user1,user2 in permutations(users_with_rating,2):
        result += [((user1[0],user2[0]),(user1[1],user2[1]))]
    return result

def calcSim(user_pair,rating_pairs):
    '''
    For each user-user pair, return the specified similarity measure,
    along with co_raters_count.
    '''
    sum_xx, sum_xy, sum_yy, sum_x, sum_y, n = (0.0, 0.0, 0.0, 0.0, 0.0, 0)

    for rating_pair in rating_pairs:
        sum_xx += np.float(rating_pair[0]) * np.float(rating_pair[0])
        sum_yy += np.float(rating_pair[1]) * np.float(rating_pair[1])
        sum_xy += np.float(rating_pair[0]) * np.float(rating_pair[1])
        # sum_y += rt[1]
        # sum_x += rt[0]
        n += 1

    cos_sim = cosine(sum_xy,np.sqrt(sum_xx),np.sqrt(sum_yy))
    return user_pair, (cos_sim,n)

shrinkage_factor_cosine = 4

def cosine(dot_product,rating_norm_squared,rating2_norm_squared):
    '''
    The cosine between two vectors A, B
       dotProduct(A, B) / (norm(A) * norm(B))
    '''
    numerator = dot_product
    denominator = rating_norm_squared * rating2_norm_squared + shrinkage_factor_cosine

    return (numerator / (float(denominator))) if denominator else 0.0

def keyOnFirstUser(user_pair,item_sim_data):
    '''
    For each user-user pair, make the first user's id the key
    '''
    (user1_id,user2_id) = user_pair
    return user1_id,(user2_id,item_sim_data)

def nearestNeighbors(user,users_and_sims,n):
    '''
    Sort the predictions list by similarity and select the top-N neighbors
    '''
    users_and_sims.sort(key=lambda x: x[1][0],reverse=True)
    return user, users_and_sims[:n]

def topNRecommendations(user_id,user_sims,users_with_rating,n):
    '''
    Calculate the top-N item recommendations for each user using the
    weighted sums method
    '''

    # initialize dicts to store the score of each individual item,
    # since an item can exist in more than one item neighborhood
    totals = defaultdict(int)
    sim_sums = defaultdict(int)
    already_voted = grouped_rates_dic[user_id]
    for (neighbor,(sim,count)) in user_sims:

        # lookup the item predictions for this neighbor
        unscored_items = users_with_rating.get(neighbor,None)

        if unscored_items:
            for (item,rating) in unscored_items:
                #if neighbor != item:

                # update totals and sim_sums with the rating data
                totals[item] += sim * rating
                sim_sums[item] += sim

    # create the normalized list of scored items
    #se voglio rating allora + media
    #users_ratings_mean[user_id]
    #/sim_sums[item]
    scored_items = [(total,item) for item,total in totals.items() if sim_sums[item] != 0 and not item in already_voted]
    #
    # sort the scored items in ascending order
    scored_items.sort(reverse=True)

    # take out the item score
    ranked_items = [x[1] for x in scored_items]
    #ranked_items = scored_items
    return user_id,ranked_items[:n]



In [10]:

def parse_content(user_ratings):
    user_id = user_ratings[0]
    ratings = user_ratings[1]
    result = list()
    for rating, item in ratings:
        result += [(user_id, item, rating)]
    return result
train_clean_data = users_final_ratings.flatMap(parse_content)
#train_clean_data = sc.parallelize([(1,1,2.5),(1,2,-1.5),(1,4,-0.5),(1,5,-0.5),(2,1,-2.6),(2,2,1.4),(2,3,-1.6),(2,4,1.4),(2,5,1.4),(3,1,-1.5),(3,3,-0.5),(3,4,1.5),(3,5,0.5),(4,1,0.25),(4,2,-0.75),(4,3,1.25),(4,4,-0.75)])



In [ ]:

train_clean_data.cache()

test_users=test_clean_data.map(lambda x: int(x[0])).collect()
#test_users = [1,2,3,4]

grouped_rates = train_clean_data.filter(lambda x: x[0] in test_users).map(lambda x: (x[0],x[1])).groupByKey().map(lambda x: (x[0], list(x[1]))).collect()
grouped_rates_dic = dict(grouped_rates)

users_ratings = train_clean_data.map(lambda x: (x[0], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))
users_ratings_mean = dict(users_ratings.mapValues(lambda x: (x[0] / x[1])).collect())

item_ratings = train_clean_data.map(lambda x: (x[1], x[2])).aggregateByKey((0,0), lambda x,y: (x[0] + y, x[1] + 1),lambda x,y: (x[0] + y[0], x[1] + y[1]))#.sortBy(lambda x: x[1][1], ascending=False)
shrinkage_factor = 10
item_ratings_mean = item_ratings.mapValues(lambda x: (x[0] / (x[1] + shrinkage_factor))).sortBy(lambda x: x[1], ascending = False).map(lambda x: x[0]).collect()


#Obtain the sparse item-user matrix: item_id -> ((user_1,rating),(user2,rating))

item_user_pairs = train_clean_data.map(lambda x: (x[1], (x[0], x[2] - users_ratings_mean[x[0]]))).groupByKey().map(lambda p: sampleInteractions(p[0],p[1],1700)).cache()
#item_user_pairs = train_clean_data.map(lambda x: (x[1], (x[0], x[2]))).groupByKey().map(lambda p: sampleInteractions(p[0],p[1],1700)).cache()

#Get all item-item pair combos: (user1_id,user2_id) -> [(rating1,rating2), (rating1,rating2),




In [ ]:

pairwise_users = item_user_pairs.filter(
    lambda p: len(p[1]) > 1).flatMap(
    lambda p: findUserPairs(p[0],p[1])).groupByKey()

#Calculate the cosine similarity for each user pair and select the top-N nearest neighbors: (user1,user2) ->    (similarity,co_raters_count)




In [ ]:
user_sims = pairwise_users.map(
    lambda p: calcSim(p[0],p[1])).map(
    lambda p: keyOnFirstUser(p[0],p[1])).groupByKey().map(
    lambda p: nearestNeighbors(p[0],list(p[1]),50))

In [ ]:
#Obtain the the item history for each user and store it as a broadcast variable user_id -> [(item_id_1, rating_1), [(item_id_2, rating_2),
user_item_hist = train_clean_data.map(lambda x: (x[0], (x[1], x[2] - users_ratings_mean[x[0]]))).groupByKey().collect()
#user_sims.collect()
ui_dict = {}
for (user,items) in user_item_hist:
    ui_dict[user] = items

uib = sc.broadcast(ui_dict)

#Calculate the top-N item recommendations for each user user_id -> [item1,item2,item3,...]
user_item_recs = user_sims.filter(lambda x: x[0] in test_users).map(
    lambda p: topNRecommendations(p[0],p[1],uib.value,5)).sortByKey().collect()

#user_item_recs[:100]

recs_dict = dict(user_item_recs)

In [34]:
def parseSubmission(line):
    user, items = line.split(",")
    items = items.split(" ")
    return (int(user), [int(item) for item in items])

submission_rdd = sc.textFile("submission.csv")
submission_header = submission_rdd.first()
submission_clean_data = submission_rdd.filter(lambda x: x != submission_header).map(parseSubmission).collect()
#submission_clean_data[:2]
submission_clean_data_dic = dict(submission_clean_data)





In [ ]:
f = open('../submission3.csv', 'wt')

writer = csv.writer(f)
writer.writerow(('userId','RecommendedItemIds'))

for u in test_users:

    predictions = recs_dict.get(u, [])
    iterator = 0
    already_voted = grouped_rates_dic[u]
    content_based_items = submission_clean_data_dic[u]
    for i in range(5 - len(predictions)):
        while (content_based_items[iterator] in already_voted) or (content_based_items[iterator] in predictions):
            iterator = iterator + 1
        predictions = predictions + [content_based_items[iterator]]
    writer.writerow((u, '{0} {1} {2} {3} {4}'.format(predictions[0], predictions[1], predictions[2], predictions[3], predictions[4])))
    #i+=1
    #print(i)

f.close()